In [59]:
import numpy as np
import sympy as sp
from qiskit.quantum_info import Operator, SparsePauliOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy

### Obtención del Hamiltoniano empleando Qiskit Nature

In [60]:
N = 2
L = 1
v_0 = 1


def well_wavefunction(n, L, x = sp.Symbol('x')):
    """
    Calcula la función de onda para un estado dado (n) en un pozo de potencial infinito.
    
    Parámetros:
        n (int): Estado cuántico del electrón.
    
    Retorna:
        La función de onda.
    """
    
    # Factor de normalización
    normalization_factor = sp.sqrt(2/L)
    
    # Función de onda radial
    well_wavefunction = normalization_factor * sp.sin(n*sp.pi*x/L)
    
    return well_wavefunction


def hamiltonian_interaction(L, N = 3, v_0 = 1):
    h_pq = []

    for i in range(1, N+1):
        h_pq.append([])
        for j in range(1, N+1):
            globals()[f'h_{i}{j}'] = -1/2 * sp.integrate(globals()[f'phi_{i}']*globals()[f'dd_phi_{j}'], (x, 0, L))
            h_pq[i-1].append(globals()[f'h_{i}{j}'])

    size = len(h_pq)
    
    g_pqrs = np.zeros((size, size, size, size))

    for i in range(N):
        for j in range(N):
            for k in range(N):
                for l in range(N):
                    globals()[f'h_{i}{j}{k}{l}'] = v_0*\
                                                   globals()[f'phi_{i+1}_1']*globals()[f'phi_{j+1}_2']*\
                                                   globals()[f'phi_{k+1}_2']*globals()[f'phi_{l+1}_1']

                    g_pqrs[i][j][k][l] = sp.integrate(globals()[f'h_{i}{j}{k}{l}'], (x_1, 0, L))

    h_pq = np.array(h_pq, dtype=float)
    g_pqrs = np.array(g_pqrs, dtype=float)

    return h_pq, g_pqrs


n_max = 8
x = sp.Symbol('x')
x_1 = sp.Symbol('x_1')

for n in range(1, n_max+1):
    for l in range(n):
        globals()[f'phi_{n}'] = well_wavefunction(n, L)
        globals()[f'dd_phi_{n}'] = sp.diff(globals()[f'phi_{n}'], x, x)

for n in range(1, n_max+1):
    for l in range(n):
        globals()[f'phi_{n}_1'] = well_wavefunction(n, L, sp.Symbol('x_1'))
        globals()[f'phi_{n}_2'] = well_wavefunction(n, L, sp.Symbol('x_1'))

    
h_pq, g_pqrs = hamiltonian_interaction(L, N, v_0)

In [61]:
h_pq

array([[ 4.9348022,  0.       ],
       [ 0.       , 19.7392088]])

In [62]:
g_pqrs

array([[[[1.5, 0. ],
         [0. , 1. ]],

        [[0. , 1. ],
         [1. , 0. ]]],


       [[[0. , 1. ],
         [1. , 0. ]],

        [[1. , 0. ],
         [0. , 1.5]]]])

In [63]:
hamiltonian = ElectronicEnergy.from_raw_integrals(h_pq, g_pqrs)

# Pasamos el hamiltoniano a terminos de operadores de qubit con el mapper
mapper = JordanWignerMapper()
fermionic_op = hamiltonian.second_q_op()
qubit_op = mapper.map(fermionic_op)

Hamiltoniano expresado en términos de las matrices de Pauli

In [64]:
qubit_op.paulis

PauliList(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IZIZ', 'YYYY',
           'XXYY', 'YYXX', 'XXXX', 'ZIIZ', 'IZZI', 'ZIZI'])

In [65]:
qubit_op.coeffs

array([ 25.924011 +0.j,  -3.0924011+0.j, -10.4946044+0.j,  -3.0924011+0.j,
       -10.4946044+0.j,   0.375    +0.j,   0.25     +0.j,   0.25     +0.j,
         0.25     +0.j,   0.25     +0.j,   0.25     +0.j,   0.25     +0.j,
         0.375    +0.j])

### Obtención del Hamiltoniano empleando directamente la segunda cuantización

In [66]:
import matrix

In [67]:
L = 1
v_0 = 1

# Creamos las matriz h_pq
h_pq = matrix.buildHPQ(L)
# We create the hpqrs matrix
g_pqrs = matrix.buildGPQRS(v_0, L)

In [68]:
h_pq

array([[ 4.9348022,  0.       ,  0.       ,  0.       ],
       [ 0.       ,  4.9348022,  0.       ,  0.       ],
       [ 0.       ,  0.       , 19.7392088,  0.       ],
       [ 0.       ,  0.       ,  0.       , 19.7392088]])

In [69]:
g_pqrs

array([[[[1.5, 0. , 0. , 0. ],
         [0. , 0. , 0. , 0. ],
         [0. , 0. , 1. , 0. ],
         [0. , 0. , 0. , 0. ]],

        [[0. , 0. , 0. , 0. ],
         [1.5, 0. , 0. , 0. ],
         [0. , 0. , 0. , 0. ],
         [0. , 0. , 1. , 0. ]],

        [[0. , 0. , 1. , 0. ],
         [0. , 0. , 0. , 0. ],
         [1. , 0. , 0. , 0. ],
         [0. , 0. , 0. , 0. ]],

        [[0. , 0. , 0. , 0. ],
         [0. , 0. , 1. , 0. ],
         [0. , 0. , 0. , 0. ],
         [1. , 0. , 0. , 0. ]]],


       [[[0. , 1.5, 0. , 0. ],
         [0. , 0. , 0. , 0. ],
         [0. , 0. , 0. , 1. ],
         [0. , 0. , 0. , 0. ]],

        [[0. , 0. , 0. , 0. ],
         [0. , 1.5, 0. , 0. ],
         [0. , 0. , 0. , 0. ],
         [0. , 0. , 0. , 1. ]],

        [[0. , 0. , 0. , 1. ],
         [0. , 0. , 0. , 0. ],
         [0. , 1. , 0. , 0. ],
         [0. , 0. , 0. , 0. ]],

        [[0. , 0. , 0. , 0. ],
         [0. , 0. , 0. , 1. ],
         [0. , 0. , 0. , 0. ],
         [0. , 1. , 0. 

In [70]:
# Creamos los operadores de creación y destrucción
A, Ap = matrix.LcreateOperators()

# Creamos el Hamiltoniano de una partícula 
Hsingle = matrix.LbuildSingleParticleH(h_pq, A, Ap)
    
# Creamos el Hamiltoniano de dos partículas 
Hdouble = matrix.LbuildDoubleParticleH(g_pqrs, A, Ap)
H = Hsingle + 1/2 * Hdouble

In [71]:
sp.Matrix(H)

Matrix([
[54.3480220054468,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0, 0],
[               0, 32.1088132032681,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0, 0],
[               0,                0, 32.1088132032681,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0, 0],
[               0,                0,                0, 11.3696044010894,                0,                0,                0,                0,                0,    

In [72]:
# Crear un operador a partir del Hamiltoniano
operator = Operator(np.flip(H))
qubit_op = SparsePauliOp.from_operator(operator)

In [73]:
qubit_op.paulis

PauliList(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZZI', 'XXYY',
           'XYYX', 'YXXY', 'YYXX', 'ZIII', 'ZIIZ', 'ZZII'])

In [74]:
qubit_op.coeffs

array([ 25.924011 +0.j, -10.4946044+0.j, -10.4946044+0.j,   0.375    +0.j,
        -3.0924011+0.j,   0.25     +0.j,  -0.25     +0.j,   0.25     -0.j,
         0.25     -0.j,  -0.25     +0.j,  -3.0924011+0.j,   0.25     +0.j,
         0.375    +0.j])